In [2]:
import pandas as pd

# Specify the file name
file_name = "한국환경공단_도로 재비산먼지 측정 정보_20241130.csv"

# Try reading the file with a different encoding
try:
    data = pd.read_csv(file_name, encoding="euc-kr")  # Adjust to "cp949" if needed
    print(data.head())
except UnicodeDecodeError as e:
    print(f"Error reading the file: {e}")
    print("Trying with a different encoding...")
    data = pd.read_csv(file_name, encoding="cp949")
    print(data.head())



         측정일자   측정시간  지역 지역명    도로명                    시작점  \
0  2024-11-01  16:04  인천  서구    에코로  인천광역시 서구 경서동 519-44 답   
1  2024-11-01  15:55  인천  서구   청라대로  인천광역시 서구 원창동 403-43 전   
2  2024-11-01  15:40  인천  서구    경제로      인천광역시 서구 연희동 821도   
3  2024-11-01  15:29  인천  서구  청라한울로    인천광역시 서구 경서동 855-5도   
4  2024-11-01  15:20  인천  서구  크리스탈로    인천광역시 서구 경서동 970-6도   

                      종점  기온  습도  재비산먼지 평균농도  오염범례  
0  인천광역시 서구 경서동 542-37 답  18  64          78    좋음  
1           인천광역시 서구 경서동  19  63          83    좋음  
2   인천광역시 서구 경서동 584-1 잡  19  63          15  매우좋음  
3    인천광역시 서구 경서동 837-8도  19  63          24  매우좋음  
4   인천광역시 서구 경서동 836-99도  19  64          37  매우좋음  
